In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import torch
import numpy
import jax

from moshi.moshi.quantization.vq import ResidualVectorQuantizer
from moshi_jax.moshi_jax.quantization.vq import ResidualVectorQuantizer as JAXQuantizer

_seanet_kwargs = {
    "channels": 1,
    "dimension": 512,
    "causal": True,
    "n_filters": 64,
    "n_residual_layers": 1,
    "activation": "ELU",
    "compress": 2,
    "dilation_base": 2,
    "disable_norm_outer_blocks": 0,
    "kernel_size": 7,
    "residual_kernel_size": 3,
    "last_kernel_size": 3,
    # We train using weight_norm but then the weights are pre-processed for inference so
    # that we can use a normal convolution.
    "norm": "none",
    "pad_mode": "constant",
    "ratios": [8, 6, 5, 4],
    "true_skip": True,
}

_quantizer_kwargs = {
    "dimension": 128,
    "n_q": 2,
    "bins": 512,
    "input_dimension": _seanet_kwargs["dimension"],
    "output_dimension": _seanet_kwargs["dimension"],
}

device = torch.get_default_device()

quantizer = ResidualVectorQuantizer(
    **_quantizer_kwargs,
)
jax_quantizer = JAXQuantizer(
    **_quantizer_kwargs,
key=jax.random.key(1))

their_params = {key: jax.numpy.array(numpy.array(value.detach())) for key, value in quantizer.named_parameters()}

/tmp/ipykernel_135508/3243001323.py:47: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  their_params = {key: jax.numpy.array(numpy.array(value.detach())) for key, value in quantizer.named_parameters()}


In [3]:
our_x = jax.random.normal(jax.random.key(1), shape=(10, 128))
jax_quantizer.vq.layers[0]._codebook._quantize(our_x)

Array([433, 128, 392,   3, 300, 476, 283, 402, 300,  88], dtype=int32)

In [4]:
our_x = jax.random.normal(jax.random.key(1), shape=(1, 512, 10))
their_x = torch.from_numpy(numpy.array(our_x))

their_result = quantizer(their_x, 10)

result =jax.vmap(jax_quantizer, in_axes=(0, None))(our_x, 10)

T after input_proj: torch.Size([1, 128, 10])
T during resquant: torch.Size([1, 128, 10])
T shape pre input: torch.Size([1, 10, 128])
T shape pre out: torch.Size([10])
T shape post out: torch.Size([1, 10])
T during resquant: torch.Size([1, 128, 10])
T shape pre input: torch.Size([1, 10, 128])
T shape pre out: torch.Size([10])
T shape post out: torch.Size([1, 10])
O after input_proj: (128, 10)
O during resquant: (128, 10)
O shape pre input: (10, 128)
O shape after out: (10,)
O during resquant: (128, 10)
